crewai 捨棄要錢的 openAI，改使用免費的 Ollama 或 gemini pro

Ollama 現階段還是不夠強，勉強可以用 openhermes，否則就只能用 gemini pro

crewai 必須在建立 agent、task 時就把角色、任務設定好，才能執行，

phidata 則可以透過 cli 交互命令行來交付任務，比較有彈性。

autogen 則是預設就有提供執行 python 程式碼的功能，可以自動產生程式碼。


In [1]:
!pip install --q crewai
!pip install --q -U duckduckgo-search
!pip install --q langchain_google_genai

In [2]:
# 匯入依賴庫
import os
from crewai import Agent, Task, Crew, Process


In [3]:
# 設定4個 LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI
# crewai 使用 Ollama 的方法：windows 只需要打開 ubuntu 視窗即可，不需要執行 litellm；Mac 則甚至不需要啟動任何程式，只要有安裝好 Ollama 即可調用。
# 與 autogen 不同，autogen 要調用 Ollama 需要執行 litellm，多一道步驟，比較麻煩
gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.7),
# Ollama 大模型，可以看要用哪一個
ollama = Ollama(model="openhermes_assistant")  # 這是用 modelfile 建立的
ollama_openhermes = Ollama(model="openhermes")

gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.6,
                             google_api_key = os.environ["GOOGLE_API_KEY"]
                             )

In [4]:
# 設定工具 tools
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search
# 最簡易型的工具定義方式
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

# 從 tools 模組中引入需要的工具
from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.file_tools import FileTools
from tools.search_tools import SearchTools
from tools.sec_tools import SECTools
# 下面添加 tools 時，必須用 類別.方法 加入工具
# BrowserTools.scrape_and_summarize_website
# CalculatorTools.calculate
# FileTools.write_file
# SearchTools.search_internet、SearchTools.search_news
# SECTools.search_10q、SECTools.search_10k
# 下面的是依樣畫葫蘆自行加入2行定義類別，親測可以匯入
from tools.find_papers_arxiv import FindPapersArxiv
from tools.get_top_hackernews_stories import GetTopHackernewsStories
# 然後就可以用下面的函式來使用工具，已測試可以~~~
# FindPapersArxiv.search_arxiv
# GetTopHackernewsStories.get_top_hackernews_stories

In [5]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[
        search_tool, 
        BrowserTools.scrape_and_summarize_website, 
        CalculatorTools.calculate, 
        FileTools.write_file,
        SearchTools.search_internet, SearchTools.search_news,
        SECTools.search_10q, SECTools.search_10k,
        FindPapersArxiv.search_arxiv,
        GetTopHackernewsStories.get_top_hackernews_stories
      ],
  # You can pass an optional llm attribute specifying what mode you wanna use.
  # It can be a local model through Ollama / LM Studio or a remote
  # model like OpenAI, Mistral, Antrophic of others (https://python.langchain.com/docs/integrations/llms/)
  #
  # Examples:
  # llm=ollama # was defined above in the file
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7)
  # llm = gemini  #using google gemini pro API
  # 或在定義後，使用 researcher.llm = ollama 或 gemini 來更改模型
)

In [6]:
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for
  your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=False,
  tools=[
        search_tool, 
        BrowserTools.scrape_and_summarize_website, 
        CalculatorTools.calculate, 
        FileTools.write_file,
        SearchTools.search_internet, SearchTools.search_news,
        SECTools.search_10q, SECTools.search_10k,
        FindPapersArxiv.search_arxiv,
        GetTopHackernewsStories.get_top_hackernews_stories
      ],
  # llm = gemini  #using google gemini pro API
)


In [7]:
# 上面可以先不指定用哪一個 llm，這裡再指定就可以
researcher.llm = gemini
writer.llm = ollama

In [8]:
# Create tasks for your agents
# 跟 phidata 比較，phidata 有提供 cli 命令交互式的方法，透過交互式的方式，可以讓使用者在命令列輸入文字，並回傳答案。
# 但 crewai 必須在程式碼中定義好要執行的任務 tasks，並執行任務。
# 要 AI 解決任務，就必須透過程式碼內輸入 tasks 的任務內容，並執行程式，才能得到結果。
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  agent=researcher
)
task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 5 paragraphs.""",
  agent=writer
)

In [9]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

In [10]:
# crew
# Get your crew to work!
result = crew.kickoff()
print(result)


[DEBUG]: Working Agent: Senior Research Analyst
[INFO]: Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search the internet
Action Input: latest advancements in AI in 2024Tool search the internet has been used.
 

Title: 10 top AI and machine learning trends for 2024 | TechTarget
Link: https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends
Snippet: 10 top AI and machine learning trends for 2024 · 1. Multimodal AI · 2. Agentic AI · 3. Open source AI · 4. Retrieval-augmented generation · 5.

-----------------
Title: What's next for AI in 2024 - MIT Technology Review
Link: https://www.technologyreview.com/2024/01/04/1086046/whats-next-for-ai-in-2024/
Snippet: In 2024, generative AI might actually 

KeyboardInterrupt: 